In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen

import time
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
ship_dir = '../../data/airbus_ship_detection/'
ship_dir_thibault = '../../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir_thibault, 'train')# Images for training
test_image_dir = os.path.join(ship_dir_thibault, 'test')# Images for testing
label_dir = os.path.join(ship_dir_thibault, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [3]:
print(len(masks.ImageId))
print(len(masks.EncodedPixels))

231723
231723


## Remove not found images

In [4]:
num_examples = len(masks.ImageId)
t_per_file = []
start = time.time()
list_error_index = []
for index, imgId in enumerate(masks.ImageId) :
    if index%100==0 or index==num_examples-1:
        clear_output(wait=True)
        t_per_file.append((time.time() - start) / 100)
        eta = np.mean(t_per_file) * (num_examples - index)
        print("ETA:", round(eta/60,2), end=" minutes ==>  ")
        print(index, "/", num_examples, end=" : ")
        print(len(list_error_index), "errors detected")
        start = time.time()
    img_path = os.path.join(train_image_dir, imgId)
    try:
        img = imread(img_path)
    except:
        list_error_index.append(index)

ETA: 0.0 minutes ==>  231722 / 231723 : 2 errors detected


In [5]:
new_dataframe = masks.drop(list_error_index, axis=0)
print("new length =", len(new_dataframe))
print("diff length :", len(masks)-len(new_dataframe))
print("errors length :", len(list_error_index))

new length = 231721
diff length : 2
errors length : 2


In [7]:
new_dataframe.to_csv(ship_dir_thibault + "train_ship_segmentations_v2_clean.csv", index=False)

## Removes duplicates

In [8]:
label_dir = os.path.join(ship_dir_thibault, 'train_ship_segmentations_v2_clean.csv')# Images for testing
dataframe = pd.read_csv(label_dir, engine="python") # Markers for ships

In [9]:
dataframe.head(5)

,ImageId,EncodedPixels
0,0001124c7.jpg,NaN
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
2,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
3,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...
4,000194a2d.jpg,198320 10 199088 10 199856 10 200624 10 201392...


In [10]:
def encodedPixel2shape(x):
    x_str = str(x)
    if x_str != 'nan':
        shape = np.array(x_str.split(" ")).astype(int)[1:][::2]
    else:
        shape = 'nan'
    return str(shape).replace(",", " ").replace("[", "").replace("]", "").replace("\n", "")

In [11]:
encodedPixel2shape(dataframe['EncodedPixels'][2])

' 1  4  5  8 10 10  9 10 10  9  9 10 10  9  9  9 10  9  9  7  6  3  1'

In [12]:
dataframe["shape"] = dataframe['EncodedPixels'].map(encodedPixel2shape)

In [13]:
dataframe.head(5)

,ImageId,EncodedPixels,shape
0,0001124c7.jpg,NaN,nan
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,17 33 33 33 33 33 33 33 33 33 33 33 33 33 33 3...
2,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...,1 4 5 8 10 10 9 10 10 9 9 10 10 9 9 ...
3,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...,9 9 9 9 9 7 7 7 7 7 7 7 9 9 9 9 9 9
4,000194a2d.jpg,198320 10 199088 10 199856 10 200624 10 201392...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...


In [14]:
dataframe_noduplicates = dataframe.drop_duplicates("shape")

In [15]:
len(dataframe)

231721

In [16]:
len(dataframe_noduplicates)

17425

In [17]:
final_dataframe = dataframe_noduplicates[["ImageId", "EncodedPixels"]][1:]

In [18]:
final_dataframe.head(10)

,ImageId,EncodedPixels
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
2,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
3,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...
4,000194a2d.jpg,198320 10 199088 10 199856 10 200624 10 201392...
5,000194a2d.jpg,55683 1 56451 1 57219 1 57987 1 58755 1 59523 ...
6,000194a2d.jpg,254389 9 255157 17 255925 17 256693 17 257461 ...
8,00021ddc3.jpg,108287 1 109054 3 109821 4 110588 5 111356 5 1...
9,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...
10,00021ddc3.jpg,74441 3 75207 5 75975 5 76743 5 77511 5 78280 ...
11,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...


In [20]:
new_dataframe.to_csv(ship_dir_thibault + "train_ship_segmentations_v2_clean_nodup.csv", index=False)